In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import scipy.stats as st
import scipy.optimize as optimize
from tqdm import tqdm  

# Initialize 
i = 1j    # imag unit
close_hour = '10'

# --- Characteristic functions ---
def chf_merton(u, r, tau, theta):
    sigma, xi, muJ, sigmaJ = theta
    omega_bar = xi * (np.exp(muJ + 0.5*sigmaJ**2) - 1)
    mu = r - 0.5*sigma**2 - omega_bar
    return np.exp(i*u*mu*tau-0.5*sigma**2*u**2*tau +
                  xi*tau*(np.exp(i*muJ*u - 0.5*sigmaJ**2*u**2) - 1))

def chf_kou(u, r, tau, theta):
    sigma, xi, alpha1, alpha2, p1 = theta
    p2 = 1 - p1
    omega_bar = xi * (1 - (p1*alpha1)/(alpha1-1) - (p2*alpha2)/(alpha2+1))
    mu = r - 0.5*sigma**2 + omega_bar
    return np.exp(i*u*mu*tau-0.5*sigma**2*u**2*tau + xi*tau*((p1*alpha1)/(alpha1 - i*u) + (p2*alpha2)/(alpha2 + i*u) - 1))

def chf_heston(u, r, tau, theta):
    v0, v_bar, gamma, rho = theta
    kappa = 0.5 # Fix kappa
    d = np.sqrt(np.power(kappa-gamma*rho*i*u,2)+(u**2+i*u)*gamma**2)
    g  = (kappa-gamma*rho*i*u-d)/(kappa-gamma*rho*i*u+d)
    C  = (1.0-np.exp(-d*tau))/(gamma*gamma*(1.0-g*np.exp(-d*tau)))\
        *(kappa-gamma*rho*i*u-d)
    A  = r * i*u *tau + kappa*v_bar*tau/gamma/gamma *(kappa-gamma*rho*i*u-d)\
        - 2*kappa*v_bar/gamma/gamma*np.log((1.0-g*np.exp(-d*tau))/(1.0-g))
    return np.exp(A + C*v0) 

def chf_bates(u, r, tau, theta):
    kappa, vbar, gamma, rho, v0, xi, muJ, sigmaJ = theta
    d = np.sqrt(np.power(kappa-gamma*rho*i*u,2)+((u**2)+i*u)*(gamma**2))
    g  = (kappa-gamma*rho*i*u-d)/(kappa-gamma*rho*i*u+d)
    C  = (1.0-np.exp(-d*tau))/((gamma**2)*(1.0-g*np.exp(-d*tau)))*(kappa-gamma*rho*i*u-d)
    AHes = r * i*u *tau + kappa*vbar*tau/gamma/gamma *(kappa-gamma*\
        rho*i*u-d) - 2*kappa*vbar/gamma/gamma*np.log((1.0-g*np.exp(-d*tau))/(1.0-g))
    A = AHes - xi * i * u * tau *(np.exp(muJ+0.5*sigmaJ**2) - 1.0) + \
            xi * tau * (np.exp(i*u*muJ - 0.5 * sigmaJ**2 * u**2) - 1.0)
    return np.exp(A + C*v0) 

# --- Cumulants ---
def merton_cumulants(tau, r, theta):
    sigma, xi, muJ, sigmaJ = theta
    omega_bar = xi * (np.exp(muJ + 0.5 * sigmaJ**2) - 1)
    c1 = tau * (r - omega_bar - 0.5 * sigma**2 - xi * muJ)
    c2 = tau * (sigma**2 + xi * (muJ**2 + sigmaJ**2))
    c4 = tau * xi * (muJ**4 + 6 * muJ**2 * sigmaJ**2 + 3 * sigmaJ**4 * xi)
    return c1, c2, c4

def kou_cumulants(tau, r, theta):
    sigma, xi, alpha1, alpha2, p1 = theta
    p2 = 1 - p1
    omega_bar = xi * (1 - (p1*alpha1)/(alpha1-1) - (p2*alpha2)/(alpha2+1))
    c1 = tau * (r + omega_bar - 0.5 * sigma**2 + ((xi * p1)/alpha1 - (xi * p2)/alpha2))
    c2 = tau * (sigma**2 + 2 * (xi * p1)/ alpha1**2 + 2 * (xi * p2)/alpha2**2)
    c4 = 24 * tau * xi * (p1 / alpha1**4 + p2 / alpha2**4) 
    return c1, c2, c4

# --- COS methods ---
def chi_psi_vec(k, a, b, c, d):
    omega = k * np.pi / (b - a)
    xi, psi = np.zeros_like(omega), np.zeros_like(omega)
    xi[0, :]  = np.exp(d) - np.exp(c)
    psi[0, :] = d - c
    if k.shape[0] > 1:
        omega_nz, k_nz = omega[1:, :], k[1:, :]
        denom = 1.0 + omega_nz**2
        xi[1:, :] = ( np.cos(omega_nz*(d-a))*np.exp(d) + omega_nz*np.sin(omega_nz*(d-a))*np.exp(d) - np.cos(omega_nz*(c-a))*np.exp(c) - omega_nz*np.sin(omega_nz*(c-a))*np.exp(c) ) / denom
        psi[1:, :] = ((b - a) / (k_nz * np.pi)) * (np.sin(omega_nz*(d - a)) - np.sin(omega_nz*(c - a)))
    return xi, psi

def payoff_coefficients_vec(CP, k, a, b):
    CP, is_call = np.asarray(CP, dtype=str), (CP == 'call')
    c = np.where(is_call, 0.0, a)
    d = np.where(is_call, b,   0.0)
    s = np.where(is_call, 1.0, -1.0)
    xi, psi = chi_psi_vec(k, a, b, c, d)
    H_k = (2.0 / (b - a)) * s * (xi - psi)
    return H_k

def delta_gamma(model, CP, S, K, tau, theta, N=512, L=10):
    r = 0.0
    S, tau, CP = np.asarray(S), np.asarray(tau), np.asarray(CP)
    x   = np.log(S / K)
    k = np.arange(N, dtype=float)[:, None]

    # Get the ChF for the model
    if model == 'merton':
        c1, c2, c4 = merton_cumulants(tau, r, theta)
        print(f"Merton c1 (drift) = {c1}") # This will likely be a non-zero number
        a = (x + c1 - L*np.sqrt(c2 + np.sqrt(c4)))[None, :]
        b = (x + c1 + L*np.sqrt(c2 + np.sqrt(c4)))[None, :]
        u = k * np.pi / (b - a)
        phi = chf_merton(u, r, tau, theta)

    elif model == 'kou':
        c1, c2, c4 = kou_cumulants(tau, r, theta)
        print(f"Kou c1 (drift) = {c1}") # This will likely be very close to zero
        a = (x + c1 - L*np.sqrt(c2 + np.sqrt(c4)))[None, :]
        b = (x + c1 + L*np.sqrt(c2 + np.sqrt(c4)))[None, :]
        u = k * np.pi / (b - a)
        phi = chf_kou(u, r, tau, theta)

    elif model == 'heston':
        a = (0.0 - L*np.sqrt(tau))[None, :]
        b = (0.0 + L*np.sqrt(tau))[None, :]
        u = k * np.pi / (b - a)
        phi = chf_heston(u, r, tau, theta)
    
    elif model == 'bates':
        a = (0.0 - L*np.sqrt(tau))[None, :]
        b = (0.0 + L*np.sqrt(tau))[None, :]
        u = k * np.pi / (b - a)
        phi = chf_bates(u, r, tau, theta)

    H_k = payoff_coefficients_vec(CP[None, :], k, a, b)
    
    w0 = np.real(phi * np.exp(-1j * u * (a - x)))
    w1 = np.real(phi * (i*u) * np.exp(-i * u * (a - x)))
    w2 = np.real(phi * ((i*u)**2 - i*u) * np.exp(-i * u * (a - x)))

    w0[0, :] *= 0.5
    w1[0, :] *= 0.5
    w2[0, :] *= 0.5

    prices = np.exp(-r * tau) * K * np.sum(w0 * H_k, axis=0)
    delta = (S**-1) * np.exp(-r * tau) * K * np.sum(w1 * H_k, axis=0)
    gamma = (S**-2) *  np.exp(-r * tau) * K * np.sum(w2 * H_k, axis=0)
    return prices.astype(float), delta.astype(float), gamma.astype(float)

# --- Data processing functions ---
def prep_side(df):
    out = df.copy()
    out = out.sort_values('timestamp').drop_duplicates('symbol', keep='last')
    out['opt_type'] = out['opt_type'].str.lower().str.strip()
    out['symbol'] = out['symbol'].str.strip()
    return out

if __name__ == "__main__":
    df_8am = pd.read_csv('/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/Data/calibration_data/08/btc_08_0dte_data.csv', parse_dates=['timestamp','expiry'])
    df_close = pd.read_csv(f'/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/Data/calibration_data/{close_hour}/btc_{close_hour}_0dte_data.csv', parse_dates=['timestamp','expiry'])
    
    cols_to_keep = ['timestamp','symbol','time_to_maturity','opt_type','strike','spot','bid_price','ask_price','mark_price', 'mark_iv', 'delta','gamma','vega','theta','rho','moneyness','moneyness_class']

    # Filter the columns to keep 
    df_8am = df_8am[cols_to_keep]
    df_close = df_close[cols_to_keep]

    # Parameter dictionary:
    params = {
        'merton': ['sigma', 'xi', 'muJ', 'sigmaJ'],
        'kou': ['sigma', 'xi', 'alpha1', 'alpha2', 'p1'],
        'heston': ['v0', 'v_bar', 'gamma', 'rho'],
        'bates': ['kappa', 'vbar', 'gamma', 'rho', 'v0', 'xi', 'muJ', 'sigmaJ']
    }

    for model in ['merton', 'kou', 'heston', 'bates']:
        params_df = pd.read_csv(f'/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/COS_Pricers/Data/Calibration/{model}_calibration_summary.csv', parse_dates=['date'])
        params_df['date'] = params_df['date'].dt.date

        pnl_rows = []
        counter = 0
        # Merge dataframes based on symbol
        pnls = pd.DataFrame()
        for day in sorted(df_8am['timestamp'].dt.date.unique()):
            df_8am_day = prep_side(df_8am[df_8am['timestamp'].dt.date == day])
            if close_hour in ['00', '01', '02', '03', '04', '05', '06', '07']:
                df_close_day = prep_side(df_close[df_close['timestamp'].dt.date == (day + pd.Timedelta(days=1))])
            else:
                df_close_day = prep_side(df_close[df_close['timestamp'].dt.date == day])

            # Merge the dataframes on the keys
            open = df_8am_day[['symbol','timestamp','spot','mark_price','time_to_maturity','opt_type',
                        'strike','delta','gamma','vega','theta','rho','moneyness','moneyness_class']].rename(
                    columns={'timestamp':'timestamp_open','spot':'spot_open','mark_price':'mark_price_open',
                        'time_to_maturity':'ttm_open', 'moneyness':'moneyness_open','moneyness_class':'moneyness_class_open'}
                    )
            close = df_close_day[['symbol','timestamp','spot','mark_price']].rename(
                    columns={'timestamp':'timestamp_close','spot':'spot_close','mark_price':'mark_price_close'}
                    )
            
            open_close_df = pd.merge(open, close, on='symbol', how='inner')
            if open_close_df.empty:
                counter += 1
                continue

            # Extract the parameters for the day
            params_row = params_df.loc[params_df['date'] == day]
            if params_row.empty:
                continue

            param_values = [params_row.iloc[0]['theta_' + p_name] for p_name in params[model]]            
            theta_open = np.array(param_values, dtype=float)
            
            # arrays for Greeks from the aligned frame
            CP_open   = open_close_df['opt_type'].values
            S_open    = open_close_df['spot_open'].values
            S_close    = open_close_df['spot_close'].values
            K         = open_close_df['strike'].values
            tau_open  = open_close_df['ttm_open'].values / (365*24*3600)  # years, as in extract_inputs

            prices_model, delta_model, gamma_model = delta_gamma(model, CP_open, S_open, K, tau_open, theta_open, N=512, L=10)

            # --- P&L Calculation in USD ---
            # Get option prices in BTC from data
            price_btc_open  = open_close_df['mark_price_open'].values
            price_btc_close = open_close_df['mark_price_close'].values

            # Convert option value to USD at open and close
            price_usd_open  = price_btc_open * S_open
            price_usd_close = price_btc_close * S_close

            # Calculate P&L of the option position in USD
            option_pnl_usd = price_usd_open - price_usd_close

            # Calculate P&L of the delta hedge in USD -> short the option, so long delta units of the underlying
            hedge_pnl_usd = delta_model * (S_close - S_open)

            # Calculate total P&L for the delta-hedged position
            pnl_delta_usd = option_pnl_usd - hedge_pnl_usd
            
            # Find the ATM option to use as the gamma hedging instrument
            atm_idx = open_close_df['moneyness_open'].abs().idxmin()

            # Get characteristics of the ATM hedging option
            price_atm_open_usd = price_usd_open[atm_idx]
            price_atm_close_usd = price_usd_close[atm_idx]
            delta_atm_model = delta_model[atm_idx]
            gamma_atm_model = gamma_model[atm_idx]

            w_gamma = - (gamma_model / (gamma_atm_model + 1e-9)) # Add epsilon for stability

            # Get PnL components
            portfolio_value_open = price_usd_open + w_gamma * price_atm_open_usd
            portfolio_value_close = price_usd_close + w_gamma * price_atm_close_usd
            portfolio_delta = delta_model + w_gamma * delta_atm_model

            pnl_gamma_usd = (portfolio_value_open - portfolio_delta * S_open) - \
                            (portfolio_value_close - portfolio_delta * S_close)
            
            # Stash result rows
            open_close_df = open_close_df[['symbol', 'strike', 'opt_type', 'timestamp_open', 'timestamp_close', 'moneyness_open', 'moneyness_class_open', 'delta','gamma','vega','theta','rho']].copy()
            open_close_df[f'delta_{model}'] = delta_model
            open_close_df[f'gamma_{model}'] = gamma_model
            open_close_df[f'PnL_delta_{model}'] = pnl_delta_usd
            open_close_df[f'PnL_delta_gamma_{model}'] = pnl_gamma_usd
            open_close_df[f'open_{model}'] = day
            pnl_rows.append(open_close_df)

        pnls = pd.concat(pnl_rows, ignore_index=True) if pnl_rows else pd.DataFrame()
        print(pnls.head())

        pnls.to_csv(f'/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/COS_Pricers/Hedging/Data/{model}_hedge_results.csv', index=False)

        print(f"Missing days: {counter}") 

Merton c1 (drift) = [0.0071814 0.0071814 0.0071814 0.0071814 0.0071814 0.0071814 0.0071814
 0.0071814 0.0071814 0.0071814 0.0071814 0.0071814 0.0071814]
Merton c1 (drift) = [0.00924208 0.00924208 0.00924208 0.00924208]
Merton c1 (drift) = [-0.00047548 -0.00047548 -0.00047548 -0.00047548 -0.00047548 -0.00047548
 -0.00047548 -0.00047548 -0.00047548]
Merton c1 (drift) = [0.0096708]
Merton c1 (drift) = [0.00949938 0.00949938 0.00949938 0.00949938 0.00949938 0.00949938
 0.00949938 0.00949938 0.00949938 0.00949938 0.00949938 0.00949938
 0.00949938 0.00949938 0.00949938 0.00949938 0.00949938 0.00949938]
Merton c1 (drift) = [-0.00042783 -0.00042783 -0.00042783 -0.00042783]
Merton c1 (drift) = [0.00686136 0.00686136 0.00686136 0.00686136 0.00686136]
Merton c1 (drift) = [0.00872402 0.00872402 0.00872402 0.00872402 0.00872402 0.00872402]
Merton c1 (drift) = [0.00232252 0.00232252 0.00232252 0.00232252 0.00232252 0.00232252]
Merton c1 (drift) = [0.00461905]
Merton c1 (drift) = [0.00253434 0.002534

In [2]:
# Set display options for better readability
pd.set_option('display.float_format', '{:.6f}'.format)

def median_absolute_deviation(x):
    return (x - x.median()).abs().median()

for model in ['merton', 'kou', 'heston', 'bates']:
    try:
        # 1. Load the hedging results you saved
        file_path = f'/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/COS_Pricers/Hedging/Data/{model}_hedge_results.csv'
        pnls_df = pd.read_csv(file_path)

        # Creating bins similar to the paper for consistent grouping
        bins = [-np.inf, -4.25, -3.75, -3.25, -2.75, -2.25, -1.75, -1.25, -0.75, -0.25, 0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, np.inf]
        labels = [-5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
        pnls_df['moneyness_bin'] = pd.cut(pnls_df['moneyness_open'], bins=bins, labels=labels)
        
        # 2. Define the metrics using the new custom function
        # Note: We pass the function itself to .agg()
        stats_to_calculate = {
            f'PnL_delta_{model}': ['median', median_absolute_deviation],
            f'PnL_delta_gamma_{model}': ['median', median_absolute_deviation]
        }

        # 3. Group by moneyness and calculate the statistics
        hedge_performance = pnls_df.groupby('moneyness_bin').agg(stats_to_calculate)
        
        # 4. Rename columns for clarity, matching the paper's concepts
        # The function name 'median_absolute_deviation' will be used as the column name
        hedge_performance.columns = [f'{model}_Delta_Median_PnL', f'{model}_Delta_MAD_PnL', 
                                    f'{model}_Gamma_Median_PnL', f'{model}_Gamma_MAD_PnL']

        # Drop NaN values
        hedge_performance = hedge_performance.dropna()

        # 5. Display the results
        # print("--- Hedging Performance Analysis ---")
        # print(hedge_performance)

    except FileNotFoundError:
        print(f"Error: The file was not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

    # Create a boolean mask for the conditions
    is_call = (pnls_df['opt_type'] == 'call')
    is_atm = pnls_df['moneyness_open'].between(-0.05, 0.05) # More readable range check
    atm_call_delta_mean = pnls_df[is_call & is_atm][f'delta_{model}'].mean()
    print(f"{model}:", atm_call_delta_mean)


merton: 0.5160247821889732
kou: 0.5132051339829816
heston: 0.5162199057540847
bates: 0.5131298548273799


/var/folders/j1/sv4t8szd51jgm826w2bv1vyr0000gn/T/ipykernel_76368/328893503.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  hedge_performance = pnls_df.groupby('moneyness_bin').agg(stats_to_calculate)
/var/folders/j1/sv4t8szd51jgm826w2bv1vyr0000gn/T/ipykernel_76368/328893503.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  hedge_performance = pnls_df.groupby('moneyness_bin').agg(stats_to_calculate)
/var/folders/j1/sv4t8szd51jgm826w2bv1vyr0000gn/T/ipykernel_76368/328893503.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass o